# Python and MySQL Introduction
## Tutorial & Notes
* Created by: Tony Held (tony.held@gmail.com)  
* Created on: 2021-03-10 
* Source: https://realpython.com/python-mysql/

In [2]:
# **********************************************
#     Jupyter Interactive Mode Settings
# **********************************************
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

# **********************************************
#     Allows autocomplete to work properly
# **********************************************
%config Completer.use_jedi = False

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
# usage pp.pprint(stuff)

In [4]:
def values(var, exclude_starting_with="_"):
    """Show variable attributes and values for diagnostic debugging purposes.
    
    Set exclude_starting_with to '' if you wish to include all attributes."""
    
    print(f'Variable type: {type(var)}')
    
    if exclude_starting_with:
        attrs = [i for i in dir(var) if not i.startswith(exclude_starting_with)]
    else:
        attrs = [i for i in dir(var)]
            
    for i in attrs:
        print(f'\n{i}:\n{"-"*20}\n{getattr(var, i)}')

In [5]:
import mysql.connector

In [6]:
from getpass import getpass
from mysql.connector import connect, Error

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        print(connection)
except Error as e:
    print(e)

Enter username: tony
Enter password: ········


In [14]:
connection = connect(host="localhost",
                     user=input("Enter username: "),
                     password=getpass("Enter password: "))
print(connection)

Enter username: tony
Enter password: ········


In [15]:
cursor = connection.cursor()

In [16]:
create_db_query = "CREATE DATABASE online_movie_rating"

'CREATE DATABASE online_movie_rating'

In [17]:
cursor.execute(create_db_query)

In [18]:
show_db_query = "SHOW DATABASES"

'SHOW DATABASES'

In [21]:
cursor.execute(show_db_query)

In [22]:
for db in cursor:
    print(db)

('information_schema',)
('mysql',)
('online_movie_rating',)
('performance_schema',)
('sys',)


In [23]:
print(cursor)

CMySQLCursor: SHOW DATABASES


In [24]:
values(cursor)

Variable type: <class 'mysql.connector.cursor_cext.CMySQLCursor'>

arraysize:
--------------------
1

callproc:
--------------------
<bound method CMySQLCursor.callproc of <mysql.connector.cursor_cext.CMySQLCursor object at 0x0000016025DAF4C0>>

close:
--------------------
<bound method CMySQLCursor.close of <mysql.connector.cursor_cext.CMySQLCursor object at 0x0000016025DAF4C0>>

column_names:
--------------------
('Database',)

description:
--------------------
[('Database', 253, None, None, None, None, 1, 0)]

execute:
--------------------
<bound method CMySQLCursor.execute of <mysql.connector.cursor_cext.CMySQLCursor object at 0x0000016025DAF4C0>>

executemany:
--------------------
<bound method CMySQLCursor.executemany of <mysql.connector.cursor_cext.CMySQLCursor object at 0x0000016025DAF4C0>>

fetchall:
--------------------
<bound method CMySQLCursor.fetchall of <mysql.connector.cursor_cext.CMySQLCursor object at 0x0000016025DAF4C0>>

fetchmany:
--------------------
<bound method

In [25]:
connection.close()

In [58]:
connection = connect(host="localhost",
                     user=input("Enter username: "),
                     password=getpass("Enter password: "),
                     database="online_movie_rating")

Enter username: tony
Enter password: ········


In [27]:
create_movies_table_query = """
CREATE TABLE movies(
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(100),
    release_year YEAR(4),
    genre VARCHAR(100),
    collection_in_mil INT
)
"""

'\nCREATE TABLE movies(\n    id INT AUTO_INCREMENT PRIMARY KEY,\n    title VARCHAR(100),\n    release_year YEAR(4),\n    genre VARCHAR(100),\n    collection_in_mil INT\n)\n'

In [29]:
with connection.cursor() as cursor:
    cursor.execute(create_movies_table_query)
    connection.commit()

In [30]:
create_reviewers_table_query = """
CREATE TABLE reviewers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_reviewers_table_query)
    connection.commit()

In [31]:
create_ratings_table_query = """
CREATE TABLE ratings (
    movie_id INT,
    reviewer_id INT,
    rating DECIMAL(2,1),
    FOREIGN KEY(movie_id) REFERENCES movies(id),
    FOREIGN KEY(reviewer_id) REFERENCES reviewers(id),
    PRIMARY KEY(movie_id, reviewer_id)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_ratings_table_query)
    connection.commit()

In [59]:
show_table_query = "DESCRIBE movies"

'DESCRIBE movies'

In [60]:
with connection.cursor() as cursor:
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    for row in result:
        print(row)

('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'decimal(4,1)', 'YES', '', None, '')


In [35]:
alter_table_query = """
ALTER TABLE movies
MODIFY COLUMN collection_in_mil DECIMAL(4,1)
"""
show_table_query = "DESCRIBE movies"

'DESCRIBE movies'

In [36]:
with connection.cursor() as cursor:
    cursor.execute(alter_table_query)
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    print("Movie Table Schema after alteration:")
    for row in result:
        print(row)

Movie Table Schema after alteration:
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'decimal(4,1)', 'YES', '', None, '')


In [38]:
insert_movies_query = """
INSERT INTO movies (title, release_year, genre, collection_in_mil)
VALUES
    ("Forrest Gump", 1994, "Drama", 330.2),
    ("3 Idiots", 2009, "Drama", 2.4),
    ("Eternal Sunshine of the Spotless Mind", 2004, "Drama", 34.5),
    ("Good Will Hunting", 1997, "Drama", 138.1),
    ("Skyfall", 2012, "Action", 304.6),
    ("Gladiator", 2000, "Action", 188.7),
    ("Black", 2005, "Drama", 3.0),
    ("Titanic", 1997, "Romance", 659.2),
    ("The Shawshank Redemption", 1994, "Drama",28.4),
    ("Udaan", 2010, "Drama", 1.5),
    ("Home Alone", 1990, "Comedy", 286.9),
    ("Casablanca", 1942, "Romance", 1.0),
    ("Avengers: Endgame", 2019, "Action", 858.8),
    ("Night of the Living Dead", 1968, "Horror", 2.5),
    ("The Godfather", 1972, "Crime", 135.6),
    ("Haider", 2014, "Action", 4.2),
    ("Inception", 2010, "Adventure", 293.7),
    ("Evil", 2003, "Horror", 1.3),
    ("Toy Story 4", 2019, "Animation", 434.9),
    ("Air Force One", 1997, "Drama", 138.1),
    ("The Dark Knight", 2008, "Action",535.4),
    ("Bhaag Milkha Bhaag", 2013, "Sport", 4.1),
    ("The Lion King", 1994, "Animation", 423.6),
    ("Pulp Fiction", 1994, "Crime", 108.8),
    ("Kai Po Che", 2013, "Sport", 6.0),
    ("Beasts of No Nation", 2015, "War", 1.4),
    ("Andadhun", 2018, "Thriller", 2.9),
    ("The Silence of the Lambs", 1991, "Crime", 68.2),
    ("Deadpool", 2016, "Action", 363.6),
    ("Drishyam", 2015, "Mystery", 3.0)
""";

In [39]:
with connection.cursor() as cursor:
    cursor.execute(insert_movies_query)
    connection.commit()

In [40]:
insert_reviewers_query = """
INSERT INTO reviewers
(first_name, last_name)
VALUES ( %s, %s )
"""
reviewers_records = [
    ("Chaitanya", "Baweja"),
    ("Mary", "Cooper"),
    ("John", "Wayne"),
    ("Thomas", "Stoneman"),
    ("Penny", "Hofstadter"),
    ("Mitchell", "Marsh"),
    ("Wyatt", "Skaggs"),
    ("Andre", "Veiga"),
    ("Sheldon", "Cooper"),
    ("Kimbra", "Masters"),
    ("Kat", "Dennings"),
    ("Bruce", "Wayne"),
    ("Domingo", "Cortes"),
    ("Rajesh", "Koothrappali"),
    ("Ben", "Glocker"),
    ("Mahinder", "Dhoni"),
    ("Akbar", "Khan"),
    ("Howard", "Wolowitz"),
    ("Pinkie", "Petit"),
    ("Gurkaran", "Singh"),
    ("Amy", "Farah Fowler"),
    ("Marlon", "Crafford"),
];

In [41]:
with connection.cursor() as cursor:
    cursor.executemany(insert_reviewers_query, reviewers_records)
    connection.commit()

In [42]:
insert_ratings_query = """
INSERT INTO ratings
(rating, movie_id, reviewer_id)
VALUES ( %s, %s, %s)
"""
ratings_records = [
    (6.4, 17, 5), (5.6, 19, 1), (6.3, 22, 14), (5.1, 21, 17),
    (5.0, 5, 5), (6.5, 21, 5), (8.5, 30, 13), (9.7, 6, 4),
    (8.5, 24, 12), (9.9, 14, 9), (8.7, 26, 14), (9.9, 6, 10),
    (5.1, 30, 6), (5.4, 18, 16), (6.2, 6, 20), (7.3, 21, 19),
    (8.1, 17, 18), (5.0, 7, 2), (9.8, 23, 3), (8.0, 22, 9),
    (8.5, 11, 13), (5.0, 5, 11), (5.7, 8, 2), (7.6, 25, 19),
    (5.2, 18, 15), (9.7, 13, 3), (5.8, 18, 8), (5.8, 30, 15),
    (8.4, 21, 18), (6.2, 23, 16), (7.0, 10, 18), (9.5, 30, 20),
    (8.9, 3, 19), (6.4, 12, 2), (7.8, 12, 22), (9.9, 15, 13),
    (7.5, 20, 17), (9.0, 25, 6), (8.5, 23, 2), (5.3, 30, 17),
    (6.4, 5, 10), (8.1, 5, 21), (5.7, 22, 1), (6.3, 28, 4),
    (9.8, 13, 1)
]
with connection.cursor() as cursor:
    cursor.executemany(insert_ratings_query, ratings_records)
    connection.commit()

In [61]:
select_movies_query = "SELECT * FROM movies LIMIT 0,5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

(1, 'Forrest Gump', 1994, 'Drama', Decimal('330.2'))
(2, '3 Idiots', 2009, 'Drama', Decimal('2.4'))
(3, 'Eternal Sunshine of the Spotless Mind', 2004, 'Drama', Decimal('34.5'))
(4, 'Good Will Hunting', 1997, 'Drama', Decimal('138.1'))
(5, 'Skyfall', 2012, 'Action', Decimal('304.6'))


In [62]:
select_movies_query = "SELECT * FROM movies LIMIT 5,5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

(6, 'Gladiator', 2000, 'Action', Decimal('188.7'))
(7, 'Black', 2005, 'Drama', Decimal('3.0'))
(8, 'Titanic', 1997, 'Romance', Decimal('659.2'))
(9, 'The Shawshank Redemption', 1994, 'Drama', Decimal('28.4'))
(10, 'Udaan', 2010, 'Drama', Decimal('1.5'))


In [63]:
select_movies_query = "SELECT title, release_year FROM movies LIMIT 5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

('Forrest Gump', 1994)
('3 Idiots', 2009)
('Eternal Sunshine of the Spotless Mind', 2004)
('Good Will Hunting', 1997)
('Skyfall', 2012)


In [64]:
select_movies_query = """
SELECT title, collection_in_mil
FROM movies
WHERE collection_in_mil > 300
ORDER BY collection_in_mil DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('Avengers: Endgame', Decimal('858.8'))
('Titanic', Decimal('659.2'))
('The Dark Knight', Decimal('535.4'))
('Toy Story 4', Decimal('434.9'))
('The Lion King', Decimal('423.6'))
('Deadpool', Decimal('363.6'))
('Forrest Gump', Decimal('330.2'))
('Skyfall', Decimal('304.6'))


In [67]:
select_movies_query = """
SELECT CONCAT(title, " (", release_year, ")"),
      collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
LIMIT 5
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))
('Toy Story 4 (2019)', Decimal('434.9'))
('The Lion King (1994)', Decimal('423.6'))


In [66]:
select_movies_query = """
SELECT CONCAT(title, " (", release_year, ")"),
      collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchmany(size=5):
        print(movie)
    cursor.fetchall()

('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))
('Toy Story 4 (2019)', Decimal('434.9'))
('The Lion King (1994)', Decimal('423.6'))


In [77]:
select_movies_query = """
SELECT title, 
AVG(rating) as average_rating,
COUNT(rating) as number_ratings
FROM ratings
LEFT JOIN movies
    ON movies.id = ratings.movie_id
GROUP BY movie_id
ORDER BY average_rating DESC, title
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('Night of the Living Dead', Decimal('9.90000'), 1)
('The Godfather', Decimal('9.90000'), 1)
('Avengers: Endgame', Decimal('9.75000'), 2)
('Eternal Sunshine of the Spotless Mind', Decimal('8.90000'), 1)
('Beasts of No Nation', Decimal('8.70000'), 1)
('Gladiator', Decimal('8.60000'), 3)
('Home Alone', Decimal('8.50000'), 1)
('Pulp Fiction', Decimal('8.50000'), 1)
('Kai Po Che', Decimal('8.30000'), 2)
('The Lion King', Decimal('8.16667'), 3)
('Air Force One', Decimal('7.50000'), 1)
('Inception', Decimal('7.25000'), 2)
('Casablanca', Decimal('7.10000'), 2)
('Udaan', Decimal('7.00000'), 1)
('Drishyam', Decimal('6.84000'), 5)
('The Dark Knight', Decimal('6.82500'), 4)
('Bhaag Milkha Bhaag', Decimal('6.66667'), 3)
('The Silence of the Lambs', Decimal('6.30000'), 1)
('Skyfall', Decimal('6.12500'), 4)
('Titanic', Decimal('5.70000'), 1)
('Toy Story 4', Decimal('5.60000'), 1)
('Evil', Decimal('5.46667'), 3)
('Black', Decimal('5.00000'), 1)


In [79]:
select_movies_query = """
SELECT CONCAT(first_name, " ", last_name), COUNT(rating) as num
FROM reviewers
INNER JOIN ratings
    ON reviewers.id = ratings.reviewer_id
GROUP BY reviewer_id
ORDER BY num DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)


('Mary Cooper', 4)
('Chaitanya Baweja', 3)
('Penny Hofstadter', 3)
('Domingo Cortes', 3)
('Akbar Khan', 3)
('Howard Wolowitz', 3)
('Pinkie Petit', 3)
('John Wayne', 2)
('Thomas Stoneman', 2)
('Mitchell Marsh', 2)
('Sheldon Cooper', 2)
('Kimbra Masters', 2)
('Rajesh Koothrappali', 2)
('Ben Glocker', 2)
('Mahinder Dhoni', 2)
('Gurkaran Singh', 2)
('Andre Veiga', 1)
('Kat Dennings', 1)
('Bruce Wayne', 1)
('Amy Farah Fowler', 1)
('Marlon Crafford', 1)


In [80]:
update_query = """
UPDATE reviewers
SET   last_name = "Cooper"
WHERE first_name = "Amy"
"""
with connection.cursor() as cursor:
    cursor.execute(update_query)
    connection.commit()